# Data Cleaning

The goal of the project is to analyse what it would be the **best home** for a **dog adopted** by the kennel of Barcelona, based on statistical analysis of the place where it'll go to live(square mts) and the area the property is located.

## 0.Import libraries

In [1]:
# Basic libraries
import pandas as pd
import numpy as np
import re
import time

# Googletrans
import googletrans
from googletrans import Translator
import copy

# Spacy --> Text reduction
import spacy
# import en_core_web_sm
nlp = spacy.load('es_core_news_sm')
import sys
# print(sys.version)

### 0.1 Installing new useful libraries

In [2]:
# Google trans
# !pip install googletrans

# Spacy --> Text reduction
# !pip install -U spacy

## 1.Loading Dataframes - Data Overview

Description of the original Dataframes we'll use for the analysis.

## Protectora Barcelona

https://www.protectorabcn.es/

Web-scraping of the page of the **public Kennel of Barcelona** (**Protectora de Barcelona**) to find some adoptable dogs in the city. (**PR03-project-web**)

In [3]:
# Share data between Jupyter Notebooks
%store -r adoptable_dogs

In [4]:
adoptable_dogs = pd.read_csv('input/'+ adoptable_dogs)

In [5]:
adoptable_dogs.head(2)

,Nombre,Caracteristicas,Foto,URL,Codigo,Animal,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Descripción,Estado,Años perrera
0,Gina,Curiosa y elegante,https://server03.protectorabcn.es/8935-home_de...,gina,39,Perros,Hembra,Grande,Adulto,Blanco,Mestizo,2010,02/2014,Si tuviéramos que describir a Gina con una sol...,En Adopción,4.1 years
1,Horus,Una familia con dedicación,https://server02.protectorabcn.es/8943-home_de...,base,997,Perros,Macho,Grande,Joven,Marron oscuro,Mestizo de Pastor Alemán,03/2016,09/2017,"Horus es este cruce de pastor alemán, nacido e...",En Adopción,1.5 years


In [6]:
print(f'Dimensions original DF-ProtectoraBCN: {adoptable_dogs.shape[0]} rows x {adoptable_dogs.shape[1]} columns')

Dimensions original DF-ProtectoraBCN: 44 rows x 16 columns


## ICanWalk

https://icanwalk.es/

Web-scraping of the page of **ICanWak**, a cool group of dog fans in order to find the **dog parks** of Barcelona.

In [7]:
# Share data between Jupyter Notebooks
%store -r dogs_parks

In [8]:
dogs_parks = pd.read_csv('input/'+ dogs_parks)

In [9]:
dogs_parks.head()

,Distritos,Dirección,Superficie (m2)
0,Ciutat Vella,Parc de la Ciutadella,287
1,Ciutat Vella,Parc de la Barceloneta,451
2,Ciutat Vella,Jardins de Sant Pau del Camp,227
3,l’Eixample,Jardins Montserrat,38
4,l’Eixample,Pl. Doctor Letamendi,8


In [10]:
print(f'Dimensions original DF-DogParks: {dogs_parks.shape[0]} rows x {dogs_parks.shape[1]} columns')

Dimensions original DF-DogParks: 105 rows x 3 columns


## Open Data Barcelona

https://opendata-ajuntament.barcelona.cat/en/

Downloading the dataset **'Purchase of registered properties of the city of Barcelona. Average surface'** 

(https://opendata-ajuntament.barcelona.cat/data/en/dataset/est-mercat-immobiliari-compravenda-sup)

Defining a variable called path that contains the path to the csv file **2019_comp_vend_sup_trim.csv**.

In [11]:
path_sqm = 'input/2019_comp_vend_sup_trim.csv'
sqm = pd.read_csv(path_sqm)
sqm.head()

,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Preu_mitja_habitatge,Nombre
0,2019,1,1,Ciutat Vella,1,el Raval,Total,58.9
1,2019,1,1,Ciutat Vella,2,el Barri Gòtic,Total,84.9
2,2019,1,1,Ciutat Vella,3,la Barceloneta,Total,45.7
3,2019,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Total,65.3
4,2019,1,2,Eixample,5,el Fort Pienc,Total,97.4


In [12]:
print(f'Dimensions original DF-OpenDataBCN: {sqm.shape[0]} rows x {sqm.shape[1]} columns')

Dimensions original DF-OpenDataBCN: 1168 rows x 8 columns


## 2. Data Manipulation 

Formatting columns and filtering the dataset with columns useful for the statistics analysis.

### 2.1 Protectora Barcelona

*Mission*: Automatically **extract relevant information about the dogs** from the column **'Descripción'** and select only the **columns useful for statistical analysis**.

Original Dataframe

In [13]:
# adoptable_dogs.head(2)

The column 'Descripción' of the DF contains interesting information about the dogs. We use the library 'Scapy' in order to extract automatically relevant information from texts we are interested in.

In [14]:
# Create a spacy object like nlp
num_words = adoptable_dogs['Descripción'].apply(lambda x: nlp(x))
original_col=len(num_words[0])

1. *Tokenization*. Separate the words in the text and convert these words into elements of a list.

2. *Text cleaning*. List of tokens without common and uninformative words.

3. *Normalization*. Only conventional words (no uppercases...)

In [15]:
def normalize(x):
    
    lexical_tokens = []

    for doc in nlp.pipe(x.astype('unicode').values, batch_size=50,n_threads=3):
        
        if doc.is_parsed:
            lexical_tokens.append([t.text.lower() for t in doc if len(t) > 4 if not (t.is_punct | t.is_stop)])
#             print(lexical_tokens)

        else:
            # We want to make sure that the lists of parsed results have the
            # same number of entries of the original Dataframe, so add some blanks in case the parse fails
            lexical_tokens.append(None)
    return lexical_tokens

In [16]:
adoptable_dogs['Descripción_norm'] = normalize(adoptable_dogs['Descripción'])

4. *Lemmatisation*. Grouping together the inflected forms of a word (lemma)

In [17]:
def lemmatisation(x):
    
    lemma=[]
    
    for col in nlp.pipe(x.astype('unicode').values, batch_size=50,n_threads=3):
        
        if col.is_parsed:
    #         print(tok)
                lemma.append([tok.lemma_.lower() for tok in col if not (tok.is_punct | tok.is_stop)])
        
        else:
            # We want to make sure that the lists of parsed results have the
            # same number of entries of the original Dataframe, so add some blanks in case the parse fails
            lemma.append(None)        
    return lemma

In [18]:
adoptable_dogs['Descripción_lemm']=lemmatisation(adoptable_dogs['Descripción_norm'])
lemma=adoptable_dogs['Descripción_lemm']
mod_column=len(lemma[0])

In [19]:
# adoptable_dogs.head(2)

**Good job!!**

In [20]:
print(f'Words reduction example (first row) --> original number of words:{original_col} vs reduced_words:{mod_column}')
print(f'Reduction number of words: {round((1-mod_column/original_col)*100)}%')

Words reduction example (first row) --> original number of words:283 vs reduced_words:85
Reduction number of words: 70%


We need to reduce the number of words even more if we want to extract some interesting characteristics of the dogs.

### HOME MADE-Sentiment Analysis

Creating some lists with words extracted from adoptable_dogs['Descripción_lemm'] and using them as a filter to define the characteristics of dogs based on: 
- need of **activity** (Normal Activity, Extra Activity)
- need of **company** (No Company, Company, Not suitable with dogs, cats, children)
- **behavior** (Positive behavior, Negative behavior)

In [21]:
adoptable_dogs['Descripción_lemm_join']=[','.join(x) for x in adoptable_dogs['Descripción_lemm']]
#adoptable_dogs['Descripción_lemm_join'].str.findall('(paseo)|(jugar)|(mover)')

Defining sub-categories of the group for identifying behavior patterns and creating a three new columns to save the results.

#### Target: Activity

**Normal activity** ['paseo','jugar','mover']

**Extra activity** ['potenciar','ritmar']

In [22]:
normal_activity=adoptable_dogs['Descripción_lemm_join'].str.findall('(paseo)|(jugar)|(mover)')
extra_activity=adoptable_dogs['Descripción_lemm_join'].str.findall('(potenciar)|(ritmar)')

In [23]:
adoptable_dogs['Actividad']=['Actividad Extra' if len(y)>=1 else 'Actividad Normal' if len(x)>=1 else 'Actividad Normal' for x,y in zip(normal_activity,extra_activity)]
# adoptable_dogs['Actividad']

#### Target: Company

**No Company** ['solo','reducir']

**Company** ['personar','compañía','estabilidad','sufrir']

**Not suitable with dogs, cats, children** ['perro','niño','gatos']

In [24]:
NO_company=adoptable_dogs['Descripción_lemm_join'].str.findall('(solo)|(reducir)')
company=adoptable_dogs['Descripción_lemm_join'].str.findall('(personar)|(compañía)|(estabilidad)|(sufrir)')
NOT_suitable=adoptable_dogs['Descripción_lemm_join'].str.findall('(perro)|(niño)|(gatos)') 

In [25]:
adoptable_dogs['Compañia']=['No Compañia' if len(x)>=1 else 'Compañia' if len(y)>=1 else 'No Apto con perros, gatos, niños' if len(z)>=1 else 'Compañia' for x,y,z in zip(NO_company,company,NOT_suitable)]
# adoptable_dogs['Compañia']

#### Target: Behavior

**Negative behavior** ['desconfiar', 'miedo', 'nerviosismo', 'peligroso','tímido']

In [26]:
#positive_behavior=adoptable_dogs['Descripción_lemm_join'].str.findall('(vital)|(activo)|(energía)')
negative_behavior=adoptable_dogs['Descripción_lemm_join'].str.findall('(desconfiar)|(miedo)|(nerviosismo)|(peligroso)|(tímido)')

In [27]:
adoptable_dogs['Caracter']=['Difícil' if len(x)>=1 else 'Cariñoso' for x in negative_behavior]
# adoptable_dogs['Caracter']

Creating a new table with these new features: **Actividad**, **Compañia**, **Caracter**. Depending on these features, we'll be able to define more precisely which could be the best house for adopted dogs and if it should be near to a public park.

In [28]:
adoptable_dogs.head(2)

,Nombre,Caracteristicas,Foto,URL,Codigo,Animal,Sexo,Medidas,Edad,Color,...,Entrada,Descripción,Estado,Años perrera,Descripción_norm,Descripción_lemm,Descripción_lemm_join,Actividad,Compañia,Caracter
0,Gina,Curiosa y elegante,https://server03.protectorabcn.es/8935-home_de...,gina,39,Perros,Hembra,Grande,Adulto,Blanco,...,02/2014,Si tuviéramos que describir a Gina con una sol...,En Adopción,4.1 years,"[tuviéramos, describir, palabra, cautivadora, ...","[tener, describir, palabra, cautivador, perro,...","tener,describir,palabra,cautivador,perro,encan...",Actividad Normal,Compañia,Cariñoso
1,Horus,Una familia con dedicación,https://server02.protectorabcn.es/8943-home_de...,base,997,Perros,Macho,Grande,Joven,Marron oscuro,...,09/2017,"Horus es este cruce de pastor alemán, nacido e...",En Adopción,1.5 years,"[horus, cruce, pastor, alemán, nacido, horus, ...","[horus, cruzar, pastor, alemán, nacer, horus, ...","horus,cruzar,pastor,alemán,nacer,horus,encanta...",Actividad Normal,Compañia,Difícil


Printing the summary (info) of the data and checking missing data in the dataset.

In [29]:
adoptable_dogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 22 columns):
Nombre                   44 non-null object
Caracteristicas          44 non-null object
Foto                     44 non-null object
URL                      44 non-null object
Codigo                   44 non-null int64
Animal                   44 non-null object
Sexo                     44 non-null object
Medidas                  44 non-null object
Edad                     44 non-null object
Color                    44 non-null object
Raza                     44 non-null object
Nacimiento               44 non-null object
Entrada                  44 non-null object
Descripción              44 non-null object
Estado                   44 non-null object
Años perrera             44 non-null object
Descripción_norm         44 non-null object
Descripción_lemm         44 non-null object
Descripción_lemm_join    44 non-null object
Actividad                44 non-null object
Compañia          

In [30]:
adoptable_dogs.isna().sum()

Nombre                   0
Caracteristicas          0
Foto                     0
URL                      0
Codigo                   0
Animal                   0
Sexo                     0
Medidas                  0
Edad                     0
Color                    0
Raza                     0
Nacimiento               0
Entrada                  0
Descripción              0
Estado                   0
Años perrera             0
Descripción_norm         0
Descripción_lemm         0
Descripción_lemm_join    0
Actividad                0
Compañia                 0
Caracter                 0
dtype: int64

In [31]:
print(f'Dimensions final DF-ProtectoraBCN: {adoptable_dogs.shape[0]} rows x {adoptable_dogs.shape[1]} columns')

Dimensions final DF-ProtectoraBCN: 44 rows x 22 columns


### 2.2 ICanWalk

*Mission*: **None**. The original dataframe is OK!

Original Dataframe

In [32]:
dogs_parks.tail()

,Distritos,Dirección,Superficie (m2)
100,Sarrià-Sant Gervasi,Jardins Mercè Rodoreda,88
101,Sarrià-Sant Gervasi,Turó Parc,123
102,Sarrià-Sant Gervasi,Jardins Doctor Samuel C. Hahnemann,283
103,Sarrià-Sant Gervasi,Pl. Ventura i Gassol,118
104,Sarrià-Sant Gervasi,Jardins Casa Sagnier\nJardins de Vil·la Amèlia...,-


In [33]:
dogs_parks.rename(columns={'Distritos':'Distrito','Dirección':'Parque', 'Superficie (m2)':'Superficie parque (mq)'}, inplace=True)

In [34]:
print(f'Dimensions final DF-DogParks: {dogs_parks.shape[0]} rows x {dogs_parks.shape[1]} columns')

Dimensions final DF-DogParks: 105 rows x 3 columns


Printing the summary (info) of the data and checking missing data in the dataset.

In [35]:
dogs_parks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
Distrito                  105 non-null object
Parque                    105 non-null object
Superficie parque (mq)    105 non-null object
dtypes: object(3)
memory usage: 2.6+ KB


In [36]:
dogs_parks.isna().sum()

Distrito                  0
Parque                    0
Superficie parque (mq)    0
dtype: int64

In [37]:
dogs_parks.dtypes

Distrito                  object
Parque                    object
Superficie parque (mq)    object
dtype: object

Formatting columns

In [38]:
dogs_parks['Parque'].str.replace('\n',' - ')

0                                  Parc de la Ciutadella
1                                 Parc de la Barceloneta
2                           Jardins de Sant Pau del Camp
3                                     Jardins Montserrat
4                                   Pl. Doctor Letamendi
                             ...                        
100                               Jardins Mercè Rodoreda
101                                            Turó Parc
102                   Jardins Doctor Samuel C. Hahnemann
103                                 Pl. Ventura i Gassol
104    Jardins Casa Sagnier - Jardins de Vil·la Amèli...
Name: Parque, Length: 105, dtype: object

In [39]:
dogs_parks['Superficie parque (mq)'] = (dogs_parks['Superficie parque (mq)'] !='-').astype(int)

In [40]:
dogs_parks['Superficie parque (mq)'].dtype

dtype('int32')

### 2.3 Open Data Barcelona

*Mission*: **Translate** the dataset from **Catalan** to **Spanish** and select only the **columns useful for statistical analysis**.

Original Dataframe

In [41]:
# sqm.head()

Formatting columns' names.

In [42]:
sqm.rename(columns = {'Nombre':'Superfície mitjana (m2)'}, inplace = True)
sqm.columns=sqm.columns.str.replace('_',' ').str.title()
sqm.head()

,Any,Trimestre,Codi Districte,Nom Districte,Codi Barri,Nom Barri,Preu Mitja Habitatge,Superfície Mitjana (M2)
0,2019,1,1,Ciutat Vella,1,el Raval,Total,58.9
1,2019,1,1,Ciutat Vella,2,el Barri Gòtic,Total,84.9
2,2019,1,1,Ciutat Vella,3,la Barceloneta,Total,45.7
3,2019,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",Total,65.3
4,2019,1,2,Eixample,5,el Fort Pienc,Total,97.4


In [43]:
sqm.isna().sum()

Any                          0
Trimestre                    0
Codi Districte               0
Nom Districte                0
Codi Barri                   0
Nom Barri                    0
Preu Mitja Habitatge         0
Superfície Mitjana (M2)    422
dtype: int64

The study is based on the average surface of the houses. I can delete items without the specification of the houses' surfaces because are NOT relevant.

In [44]:
sqm[sqm.isnull().any(axis=1)]

,Any,Trimestre,Codi Districte,Nom Districte,Codi Barri,Nom Barri,Preu Mitja Habitatge,Superfície Mitjana (M2)
57,2019,1,9,Sant Andreu,58,Baró de Viver,Total,NaN
78,2019,1,2,Eixample,6,la Sagrada Família,Habitatge nou lliure,NaN
82,2019,1,2,Eixample,10,Sant Antoni,Habitatge nou lliure,NaN
83,2019,1,3,Sants-Montjuïc,11,el Poble Sec,Habitatge nou lliure,NaN
84,2019,1,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Habitatge nou lliure,NaN
...,...,...,...,...,...,...,...,...
1092,2019,4,10,Sant Martí,71,Provençals del Poblenou,Habitatge nou protegit,NaN
1093,2019,4,10,Sant Martí,72,Sant Martí de Provençals,Habitatge nou protegit,NaN
1094,2019,4,10,Sant Martí,73,la Verneda i la Pau,Habitatge nou protegit,NaN
1106,2019,4,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Habitatge usat,NaN


In [45]:
sqm.dropna(inplace=True)

In [46]:
sqm.isna().sum()

Any                        0
Trimestre                  0
Codi Districte             0
Nom Districte              0
Codi Barri                 0
Nom Barri                  0
Preu Mitja Habitatge       0
Superfície Mitjana (M2)    0
dtype: int64

### API Google Translate

The dataset is in Catalan so I have to translate it into Spanish (using **Google Trans**).

Translating columns' names.

In [47]:
# Make a deep copy of the DF
sqm_ca = sqm.copy()

In [48]:
# create a translator object
translator = Translator()

Available languages for translation.

In [49]:
# print(googletrans.LANGUAGES)

In [50]:
# Translate columns' name using rename function
sqm_ca.rename(columns=lambda x: translator.translate(x, src='ca', dest='es').text, inplace=True)

# Translated column names
sqm_ca.columns=sqm_ca.columns.str.title()
sqm_ca=sqm_ca.rename(columns = {'Precio Media Vivienda':'Precio Medio Vivienda'})

In [51]:
col_name=list(sqm_ca.columns)
col_name

['Año',
 'Trimestre',
 'Código Distrito',
 'Nombre Distrito',
 'Código Barrio',
 'Nombre Barrio',
 'Precio Medio Vivienda',
 'Superficie Media (M2)']

Selecting only NUMERICAL columns and translating selected columns.

In [52]:
num_name = sqm_ca.select_dtypes(include='number')
num_name.head()

,Año,Trimestre,Código Distrito,Código Barrio,Superficie Media (M2)
0,2019,1,1,1,58.9
1,2019,1,1,2,84.9
2,2019,1,1,3,45.7
3,2019,1,1,4,65.3
4,2019,1,2,5,97.4


Selecting only CATEGORICAL columns and translating selected columns.

In [53]:
cat_name = sqm_ca.select_dtypes(exclude='number')
cat_name.head()

,Nombre Distrito,Nombre Barrio,Precio Medio Vivienda
0,Ciutat Vella,el Raval,Total
1,Ciutat Vella,el Barri Gòtic,Total
2,Ciutat Vella,la Barceloneta,Total
3,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera",Total
4,Eixample,el Fort Pienc,Total


In [54]:
sqm_district= sqm_ca[['Nombre Distrito']]
sqm_district

,Nombre Distrito
0,Ciutat Vella
1,Ciutat Vella
2,Ciutat Vella
3,Ciutat Vella
4,Eixample
...,...
1163,Sant Martí
1164,Sant Martí
1165,Sant Martí
1166,Sant Martí


In [55]:
sqm_ca= sqm_ca[['Nombre Barrio','Precio Medio Vivienda']]
sqm_ca

,Nombre Barrio,Precio Medio Vivienda
0,el Raval,Total
1,el Barri Gòtic,Total
2,la Barceloneta,Total
3,"Sant Pere, Santa Caterina i la Ribera",Total
4,el Fort Pienc,Total
...,...,...
1163,Diagonal Mar i el Front Marítim del Poblenou,Habitatge usat
1164,el Besòs i el Maresme,Habitatge usat
1165,Provençals del Poblenou,Habitatge usat
1166,Sant Martí de Provençals,Habitatge usat


In [56]:
# Translating cell values
translations = {}
for column in sqm_ca.columns:
    # unique elements of the column
    unique_elements = sqm_ca[column].unique()
    for element in unique_elements:
        # add translation to the dictionary
        translations[element] = translator.translate(element, src='ca', dest='es').text   
print(translations)

{'el Raval': 'el Raval', 'el Barri Gòtic': 'el Barrio Gótico', 'la Barceloneta': 'la Barceloneta', 'Sant Pere, Santa Caterina i la Ribera': 'San Pedro, Santa Caterina y la Ribera', 'el Fort Pienc': 'el Fort Pienc', 'la Sagrada Família': 'la Sagrada Familia', "la Dreta de l'Eixample": 'la Derecha del Eixample', "l'Antiga Esquerra de l'Eixample": 'la Antigua Izquierda del Ensanche', "la Nova Esquerra de l'Eixample": 'la Nueva Izquierda del Ensanche', 'Sant Antoni': 'San Antonio', 'el Poble Sec': 'el Poble Sec', 'la Marina del Prat Vermell': 'la Marina del Prat Vermell', 'la Marina de Port': 'la Marina de Puerto', 'la Font de la Guatlla': 'la Fuente de la Codorniz', 'Hostafrancs': 'Hostafrancs', 'la Bordeta': 'la Bordeta', 'Sants - Badal': 'Santos - Badal', 'Sants': 'Santos', 'les Corts': 'las Cortes', 'la Maternitat i Sant Ramon': 'la Maternidad y Sant Ramon', 'Pedralbes': 'Pedralbes', 'Vallvidrera, el Tibidabo i les Planes': 'Vallvidrera, el Tibidabo y las Llanuras', 'Sarrià': 'Sarrià',

Modifying wrong traslations.

In [57]:
translations['Total'] = 'Total'
translations['Vivienda nueva protegido'] = 'Vivienda nueva protegida'
translations['vivienda usada'] = 'Vivienda usada'

Modifying the original dataframe.

In [58]:
sqm_ca.replace(translations, inplace = True)

Checking the traslation.

In [59]:
sqm_ca

,Nombre Barrio,Precio Medio Vivienda
0,el Raval,Total
1,el Barrio Gótico,Total
2,la Barceloneta,Total
3,"San Pedro, Santa Caterina y la Ribera",Total
4,el Fort Pienc,Total
...,...,...
1163,Diagonal Mar i el Front Marítim del Poblenou,vivienda usada
1164,el Besòs y el Maresme,vivienda usada
1165,Provençals del Poblenou,vivienda usada
1166,Sant Martí de Provençals,vivienda usada


Creating the final dataframe translated into Spanish.

In [60]:
sqm_es = pd.concat([sqm_district, sqm_ca, num_name], axis=1)
sqm_es

,Nombre Distrito,Nombre Barrio,Precio Medio Vivienda,Año,Trimestre,Código Distrito,Código Barrio,Superficie Media (M2)
0,Ciutat Vella,el Raval,Total,2019,1,1,1,58.9
1,Ciutat Vella,el Barrio Gótico,Total,2019,1,1,2,84.9
2,Ciutat Vella,la Barceloneta,Total,2019,1,1,3,45.7
3,Ciutat Vella,"San Pedro, Santa Caterina y la Ribera",Total,2019,1,1,4,65.3
4,Eixample,el Fort Pienc,Total,2019,1,2,5,97.4
...,...,...,...,...,...,...,...,...
1163,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou,vivienda usada,2019,4,10,69,82.1
1164,Sant Martí,el Besòs y el Maresme,vivienda usada,2019,4,10,70,68.5
1165,Sant Martí,Provençals del Poblenou,vivienda usada,2019,4,10,71,75.5
1166,Sant Martí,Sant Martí de Provençals,vivienda usada,2019,4,10,72,70.6


In [61]:
final_sqm = sqm_es[['Nombre Distrito','Nombre Barrio','Superficie Media (M2)']]
final_sqm

,Nombre Distrito,Nombre Barrio,Superficie Media (M2)
0,Ciutat Vella,el Raval,58.9
1,Ciutat Vella,el Barrio Gótico,84.9
2,Ciutat Vella,la Barceloneta,45.7
3,Ciutat Vella,"San Pedro, Santa Caterina y la Ribera",65.3
4,Eixample,el Fort Pienc,97.4
...,...,...,...
1163,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou,82.1
1164,Sant Martí,el Besòs y el Maresme,68.5
1165,Sant Martí,Provençals del Poblenou,75.5
1166,Sant Martí,Sant Martí de Provençals,70.6


In [62]:
final_sqm.rename(columns={'Nombre Distrito':'Distrito','Nombre Barrio':'Barrio','Superficie Media (M2)':'Superficie media casa (mq)'}, inplace=True)

D:\ANACONDA\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [63]:
print(f'Dimensions final DF-OpenDataBCN: {final_sqm.shape[0]} rows x {final_sqm.shape[1]} columns')

Dimensions final DF-OpenDataBCN: 746 rows x 3 columns


Printing the summary (info) of the data and checking missing data in the dataset.

In [64]:
final_sqm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 746 entries, 0 to 1167
Data columns (total 3 columns):
Distrito                      746 non-null object
Barrio                        746 non-null object
Superficie media casa (mq)    746 non-null float64
dtypes: float64(1), object(2)
memory usage: 23.3+ KB


In [65]:
final_sqm.isna().sum()

Distrito                      0
Barrio                        0
Superficie media casa (mq)    0
dtype: int64

## 3.Create Dataframes for Statistical Analysis

The **final goal** is to obtain a **two dataframes** (dogs, houses) that can be used for the **Statistical Analysis**.

## 3.1 Dogs

### 3.1 Protectora Barcelona

In [66]:
print(f'Dimensions final DF-ProtectoraBCN: {adoptable_dogs.shape[0]} rows x {adoptable_dogs.shape[1]} columns')
print(f'Columns final DF-ProtectoraBCN: {list(adoptable_dogs.columns)}')

Dimensions final DF-ProtectoraBCN: 44 rows x 22 columns
Columns final DF-ProtectoraBCN: ['Nombre', 'Caracteristicas', 'Foto', 'URL', 'Codigo', 'Animal', 'Sexo', 'Medidas', 'Edad', 'Color', 'Raza', 'Nacimiento', 'Entrada', 'Descripción', 'Estado', 'Años perrera', 'Descripción_norm', 'Descripción_lemm', 'Descripción_lemm_join', 'Actividad', 'Compañia', 'Caracter']


Dropping unnecessary columns.

In [67]:
dogs_df=adoptable_dogs[['Nombre', 'Caracteristicas', 'Sexo', 'Medidas', 'Edad', 'Color', 'Raza', 
                'Nacimiento', 'Entrada', 'Años perrera', 'Estado', 'Actividad', 'Compañia', 'Caracter']]
dogs_df.head()

,Nombre,Caracteristicas,Sexo,Medidas,Edad,Color,Raza,Nacimiento,Entrada,Años perrera,Estado,Actividad,Compañia,Caracter
0,Gina,Curiosa y elegante,Hembra,Grande,Adulto,Blanco,Mestizo,2010,02/2014,4.1 years,En Adopción,Actividad Normal,Compañia,Cariñoso
1,Horus,Una familia con dedicación,Macho,Grande,Joven,Marron oscuro,Mestizo de Pastor Alemán,03/2016,09/2017,1.5 years,En Adopción,Actividad Normal,Compañia,Difícil
2,Muñeca,Preparada para volver a empezar,Hembra,Mediano,Adulto,Negro,Mestizo,02/2012,07/2018,6.4 years,En Adopción,Actividad Normal,Compañia,Cariñoso
3,Goliath,Necesita un hogar urgente,Macho,Grande,Joven,Gris,Fila Brasileiro,03/2018,02/2019,0.9 years,Urgente,Actividad Normal,"No Apto con perros, gatos, niños",Difícil
4,Bruno,Una persona que lo entienda,Macho,Pequeño,Adulto,Beig,Mestizo de Terrier,10/2011,01/2018,6.3 years,En Adopción,Actividad Normal,Compañia,Cariñoso


In [68]:
print(f'Dimensions final DF-OpenDataBCN: {dogs_df.shape[0]} rows x {dogs_df.shape[1]} columns')

Dimensions final DF-OpenDataBCN: 44 rows x 14 columns


### Summary (info)

In [69]:
dogs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 14 columns):
Nombre             44 non-null object
Caracteristicas    44 non-null object
Sexo               44 non-null object
Medidas            44 non-null object
Edad               44 non-null object
Color              44 non-null object
Raza               44 non-null object
Nacimiento         44 non-null object
Entrada            44 non-null object
Años perrera       44 non-null object
Estado             44 non-null object
Actividad          44 non-null object
Compañia           44 non-null object
Caracter           44 non-null object
dtypes: object(14)
memory usage: 4.9+ KB


## Houses

### 3.2.1 ICanWalk

In [70]:
print(f'Dimensions final DF-DogParks: {dogs_parks.shape[0]} rows x {dogs_parks.shape[1]} columns')
print(f'Columns final DF-DogParks: {list(dogs_parks.columns)}')

Dimensions final DF-DogParks: 105 rows x 3 columns
Columns final DF-DogParks: ['Distrito', 'Parque', 'Superficie parque (mq)']


### 3.2.2 Open Data Barcelona

In [71]:
print(f'Dimensions final DF-OpenDataBCN: {final_sqm.shape[0]} rows x {final_sqm.shape[1]} columns')
print(f'Columns DF-OpenDataBCN: {list(final_sqm.columns)}')

Dimensions final DF-OpenDataBCN: 746 rows x 3 columns
Columns DF-OpenDataBCN: ['Distrito', 'Barrio', 'Superficie media casa (mq)']


Formatting **District** column of the final_sqm Dataset in order to **merge** it with the dog_parks'one.

In [72]:
dogs_parks['Distrito'].unique()

array(['Ciutat Vella', 'l’Eixample', 'Gràcia', 'Horta-Guinardó',
       'Les Corts', 'Nou Barris', 'Sant Andreu', 'Sant Martí',
       'Sants-Montjuïc', 'Sarrià-Sant Gervasi'], dtype=object)

In [73]:
final_sqm['Distrito'].unique()

array(['Ciutat Vella', 'Eixample', 'Sants-Montjuïc', 'Les Corts',
       'Sarrià-Sant Gervasi', 'Gràcia', 'Horta-Guinardó', 'Nou Barris',
       'Sant Andreu', 'Sant Martí'], dtype=object)

In [74]:
final_sqm.replace({'Eixample': 'l’Eixample'}, inplace=True)

D:\ANACONDA\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [75]:
houses_df=pd.merge(dogs_parks, final_sqm, on='Distrito')
houses_df.head()

,Distrito,Parque,Superficie parque (mq),Barrio,Superficie media casa (mq)
0,Ciutat Vella,Parc de la Ciutadella,1,el Raval,58.9
1,Ciutat Vella,Parc de la Ciutadella,1,el Barrio Gótico,84.9
2,Ciutat Vella,Parc de la Ciutadella,1,la Barceloneta,45.7
3,Ciutat Vella,Parc de la Ciutadella,1,"San Pedro, Santa Caterina y la Ribera",65.3
4,Ciutat Vella,Parc de la Ciutadella,1,el Raval,59.6


In [76]:
print(f'Dimensions final merge_DF: {houses_df.shape[0]} rows x {houses_df.shape[1]} columns')

Dimensions final merge_DF: 8004 rows x 5 columns


In [77]:
houses_final = houses_df[['Distrito','Barrio','Superficie media casa (mq)','Parque','Superficie parque (mq)',]]

In [78]:
print(f'Dimensions final merge_DF: {houses_final.shape[0]} rows x {houses_final.shape[1]} columns')

Dimensions final merge_DF: 8004 rows x 5 columns


## 4.Export Dataframes

Export final dataframe in csv.

### 4.1 Dogs

In [79]:
dogs = 'dogs '+time.strftime('%d-%m-%Y_%H-%M')+'.csv'
dogs

'dogs 05-10-2020_14-54.csv'

In [81]:
dogs_df.to_csv('output_data-cleaning/'+dogs, sep=',', index=False)

In [82]:
# Share data between Jupyter Notebooks
%store dogs

Stored 'dogs' (str)


### 4.2 Houses

In [83]:
houses = 'houses+parques.csv'

In [85]:
houses_df.to_csv('output_data-cleaning/'+houses, sep=',', index=False)

In [86]:
# Share data between Jupyter Notebooks
%store houses

Stored 'houses' (str)
